# "Engine knock detection AI part 1/5"
> "This is the first notebook in a series where I'm following along with the [fast.ai](https://www.fast.ai/) course [Practical Deep Learning for Coders](https://course.fast.ai/) by training an AI to identify knocking in 4-stroke car petrol engines"

# Background
The inspiration for this project comes from the presentation of fast.ai course alumni projects by Jeremy Howard in some of the first lessons where training an image classifier using spectrograms to identify sounds is mentioned.

What this means in simpler terms is that an image classifying algorithm is presented with a series of pictures labeled with which class or case they represent. Each picture represents what that sound looks like if one were to plot the sound intensities at different frequencies as a function of time. 

# Method

A set of recordings of both knocking and healthy sounding engines has first to be aquired in order to create a training set of spectrograms. I've chosen to create two playlists on youtube of videos specifically depicting car engines, one for each case. I made sure to just include petrol engines since my suspicion was that the sound of a diesel engine might throw things off.

As a non car mechanic I made that assumption based on the idea that if I could distinguish a well running engine from a knocking one then the resulting model could do it too. I found I had some trouble making the distinction with some of the cold started diesels I listened to.

## Downloading audio files
After creating the playlists Google Drive is mounted to store the intermediary sound files.

In [ ]:
#collapse-output
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


The youtube-dl python package is installed and the soundtrack from each of the videos is downloaded.

In [ ]:
#collapse-output
!pip install --upgrade youtube-dl

     |████████████████████████████████| 1.8MB 2.8MB/s 


In [ ]:
import youtube_dl

In [ ]:
#collapse-output
ydl = youtube_dl.YoutubeDL({'outtmpl': '%(id)s%(ext)s'})

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/playlist?list=PL9R1Zswn-XPCh3A3bV9Vnf7ui9YCZWo9w'])

[youtube:playlist] PL9R1Zswn-XPCh3A3bV9Vnf7ui9YCZWo9w: Downloading webpage
[download] Downloading playlist: engine_normal
[youtube:playlist] playlist engine_normal: Downloading 15 videos
[download] Downloading video 1 of 15
[youtube] ho38ZYkQJxs: Downloading webpage
[youtube] ho38ZYkQJxs: Downloading video info webpage
[download] Destination: 2003 toyota corolla engine sound-ho38ZYkQJxs.webm
[download] 100% of 2.44MiB in 00:00
[ffmpeg] Destination: 2003 toyota corolla engine sound-ho38ZYkQJxs.mp3
Deleting original file 2003 toyota corolla engine sound-ho38ZYkQJxs.webm (pass -k to keep)
[download] Downloading video 2 of 15
[youtube] Eky3PMh76gY: Downloading webpage
[youtube] Eky3PMh76gY: Downloading video info webpage
[youtube] Eky3PMh76gY: Downloading MPD manifest
[dashsegments] Total fragments: 8
[download] Destination: 2017 Hyundai Elantra SE 2.0L 4 Cylinder - Normal Engine Running Noises-Eky3PMh76gY.webm
[download] 100% of 1.04MiB in 00:02
[ffmpeg] Destination: 2017 Hyundai Elantra 

In [ ]:
!rm -rf ./sample_data

Checking to see that all the expected files are present.

In [ ]:
!ls *.mp3

'1996 Subaru Outback, Normal Engine Sound-8BOtWY3RobA.mp3'
'2003 toyota corolla engine sound-ho38ZYkQJxs.mp3'
'2016-2018 Toyota Tacoma Loud Engine sound is Normal-nSB65kClSXM.mp3'
'2016 Kia Rio 1.6L V4 Engine - Normal Running Noises _ Sounds-nGQcC7giMkU.mp3'
'2017 Hyundai Elantra SE 2.0L 4 Cylinder - Normal Engine Running Noises-Eky3PMh76gY.mp3'
'2017 Kia Rio 1.6L 4 Cylinder GDI Engine - Normal Engine Sounds-2EQcfJAU7IM.mp3'
'2017 Nissan Versa Note - Normal Engine Running Noises - 1.6L Engine-PRO0HgD9qx4.mp3'
'2018 Hyundai Tucson 2.0 GDI Nu Engine Sound Normal-pLmX2ws7znE.mp3'
'3.2 fsi engine sound cold (normal or not)-mOFcfwLTNkY.mp3'
'Bmw f30 Noisy Engine Sound-bkPtY1pW82Y.mp3'
'Listen Toyota 2.4 VVT-i engine sound, when engine is very OK. Years 2002 to 2015-6GPgodkLSkQ.mp3'
'Mini R56S normal engine sound-8cXmy_U0_28.mp3'
'Nissan Frontier 4L v6 engine sound idle-YiwsC_UlDFY.mp3'
'Normal Engine Idle Sound Cold & Warm - 2006 Nissan Sentra-5q1uATQu8zg.mp3'
'Volvo S60 2.0T - Normal engin

Moving the sound files from the Colab instance to a folder on Google Drive.

In [ ]:
!mv *.mp3 /content/drive/My\ Drive/Colab\ Notebooks/fast.ai/KnockKnock/data/normal/

Rinse and repeat for the other playlist.

In [ ]:
#collapse-hide
#collapse-output
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/playlist?list=PL9R1Zswn-XPCnpyXQLRPYVJr4BXJUYie8'])

[youtube:playlist] PL9R1Zswn-XPCnpyXQLRPYVJr4BXJUYie8: Downloading webpage
[download] Downloading playlist: engine_knocking
[youtube:playlist] playlist engine_knocking: Downloading 15 videos
[download] Downloading video 1 of 15
[youtube] JIOdS7XbHys: Downloading webpage
[youtube] JIOdS7XbHys: Downloading video info webpage
[download] Destination: How To Diagnose An Engine Knock.MP4-JIOdS7XbHys.webm
[download] 100% of 3.87MiB in 00:00
[ffmpeg] Destination: How To Diagnose An Engine Knock.MP4-JIOdS7XbHys.mp3
Deleting original file How To Diagnose An Engine Knock.MP4-JIOdS7XbHys.webm (pass -k to keep)
[download] Downloading video 2 of 15
[youtube] gBA1zC5duuI: Downloading webpage
[youtube] gBA1zC5duuI: Downloading video info webpage
[download] Destination: Engine Noise Diagnosis-Part 1-gBA1zC5duuI.webm
[download] 100% of 17.50MiB in 00:00
[ffmpeg] Destination: Engine Noise Diagnosis-Part 1-gBA1zC5duuI.mp3
Deleting original file Engine Noise Diagnosis-Part 1-gBA1zC5duuI.webm (pass -k to ke

In [ ]:
#collapse-hide
!mv *.mp3 /content/drive/My\ Drive/Colab\ Notebooks/fast.ai/KnockKnock/data/knocking/

## Slicing audio files
The 15 mp3 files were then sliced into 10 second clips in [Audacity](https://www.audacityteam.org/). This could probably have been done programmatically but as trivial as the task initally may sound there would most likely have been complications. The videos do not only show the car running with the hood popped up. There's some talking, explaining and fumbling around in quite a few of them. 
